In [1]:
import pandas as pd
import numpy as np
import geopy.distance
from sklearn.cluster import DBSCAN
import sys
from sklearn.model_selection import train_test_split

In [14]:
ccInfo = pd.read_csv('./data/cc_info.csv')
transaction = pd.read_csv('./data/transactions.csv')
df = pd.merge(ccInfo, transaction, on='credit_card', how='outer')

In [15]:
df.credit_card_limit = df.credit_card_limit.astype('float64')

In [17]:
# Assuming df is your merged dataframe
# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Extract hour and day of the week from the 'date' column
df['transaction_hour'] = df['date'].dt.hour
df['transaction_day_of_week'] = df['date'].dt.dayofweek

# Compute transaction amount relative to credit card limit
df['transaction_amount_relative_to_limit'] = df['transaction_dollar_amount'] / df['credit_card_limit']

# Create new columns to store the results
df['transaction_frequency'] = 0
df['avg_transaction_amount_24h'] = 0.0

# Loop through each unique credit card
for card in df['credit_card'].unique():
    # Subset the DataFrame to only transactions for this card
    card_df = df[df['credit_card'] == card]

    # Sort by date
    card_df = card_df.sort_values(by='date')

    # Calculate time difference between each transaction in hours
    time_diffs = card_df['date'].diff().dt.total_seconds() / 3600  # Convert to hours

    # Initialize rolling count, sum, and time differences
    rolling_count = np.array([1])
    rolling_sum = np.array([card_df.iloc[0]['transaction_dollar_amount']])
    rolling_time_diffs = np.array([0.0])

    # Loop through each transaction
    for i in range(1, len(card_df)):
        # Get the time difference to the current transaction
        time_diff = time_diffs.iloc[i]

        # Add the current transaction to the rolling count and sum
        rolling_count = np.append(rolling_count, rolling_count[-1] + 1)
        rolling_sum = np.append(rolling_sum, rolling_sum[-1] + card_df.iloc[i]['transaction_dollar_amount'])
        rolling_time_diffs = np.append(rolling_time_diffs, time_diff + rolling_time_diffs[-1])

        # Remove transactions older than 24 hours from the rolling count and sum
        while rolling_time_diffs[0] > 24:
            rolling_count[-1] = rolling_count[-1] - 1
            rolling_sum[-1] = rolling_sum[-1] - card_df.iloc[i - len(rolling_time_diffs)]['transaction_dollar_amount']
            rolling_time_diffs = rolling_time_diffs[1:]

    # Store the results back in the main DataFrame
    df.loc[card_df.index, 'transaction_frequency'] = rolling_count
    df.loc[card_df.index, 'avg_transaction_amount_24h'] = rolling_sum / rolling_count

In [19]:
# Initialize DBSCAN
db = DBSCAN(eps=0.3, min_samples=10)

# Loop through each unique credit card
for card in df['credit_card'].unique():
    # Subset the DataFrame to only transactions for this card
    card_df = df[df['credit_card'] == card]

    # Perform DBSCAN on longitude and latitude
    coordinates = card_df[['Lat', 'Long']]
    db.fit(coordinates)

    # Get the cluster labels
    labels = db.labels_

    if len(set(labels)) > 1:  # There is at least one cluster
        # Calculate the centroid of the largest cluster
        largest_cluster = np.argmax(np.bincount(labels[labels != -1]))
        centroid = np.mean(coordinates[labels == largest_cluster], axis=0)

        # Calculate the distance to the centroid for each transaction
        distances = []
        for i in range(len(card_df)):
            coords_1 = (card_df.iloc[i]['Lat'], card_df.iloc[i]['Long'])
            coords_2 = (centroid[0], centroid[1])
            distances.append(geopy.distance.distance(coords_1, coords_2).km)
    else:  # No valid cluster
        distances = [sys.float_info.max] * len(card_df)

    # Store the results back in the main DataFrame
    df.loc[card_df.index, 'distance_from_common_location'] = distances

In [20]:
df.to_csv('./data/pre-processed-raw.csv', index=False)

In [8]:
df = df.drop(columns=['city', 'state', 'zipcode', 'date', 'Long', 'Lat', 'transaction_dollar_amount'])

In [13]:
df.to_csv('./data/pre-processed-clean.csv', index=False)

In [ ]:
train, test = train_test_split(df, test_size=0.2)

In [ ]:
len(train), len(test)